In [25]:
import numpy as np 
import pandas as pd 
from scipy.spatial.distance import correlation
import itertools 

In [5]:
dataFile='/Users/harshilyadav/Desktop/ml-100k/u.data'
data=pd.read_csv(dataFile,sep="\t",header=None,
                 names=['userId','itemId','rating','timestamp'])
data.head()

,userId,itemId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
movieInfoFile="/Users/harshilyadav/Desktop/ml-100k/u.item"
movieInfo=pd.read_csv(movieInfoFile,sep="|", header=None, index_col=False,
                      names=["itemId","title"], usecols=[0,1],encoding = 'latin')
movieInfo.head()

,itemId,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [6]:
data=pd.merge(data,movieInfo,left_on='itemId',right_on="itemId")
data.head()

,userId,itemId,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [7]:
userIds=data.userId 
userIds2=data[['userId']]

In [8]:
userIds.head()


0    196
1     63
2    226
3    154
4    306
Name: userId, dtype: int64

In [9]:
userIds2.head()


,userId
0,196
1,63
2,226
3,154
4,306


In [10]:
type(userIds)


pandas.core.series.Series

In [11]:
type(userIds2)

pandas.core.frame.DataFrame

In [12]:
data.loc[0:10,['userId']]

,userId
0,196
1,63
2,226
3,154
4,306
5,296
6,34
7,271
8,201
9,209


In [13]:
toyStoryUsers=data[data.title=="Toy Story (1995)"]
toyStoryUsers.head()

,userId,itemId,rating,timestamp,title
3397,308,1,4,887736532,Toy Story (1995)
3398,287,1,5,875334088,Toy Story (1995)
3399,148,1,4,877019411,Toy Story (1995)
3400,280,1,4,891700426,Toy Story (1995)
3401,66,1,3,883601324,Toy Story (1995)


In [14]:
data=pd.DataFrame.sort_values(data,['userId','itemId'],ascending=[0,1])

numUsers=max(data.userId)
numMovies=max(data.itemId)

moviesPerUser=data.userId.value_counts()
usersPerMovie=data.title.value_counts()

usersPerMovie

Star Wars (1977)                                     583
Contact (1997)                                       509
Fargo (1996)                                         508
Return of the Jedi (1983)                            507
Liar Liar (1997)                                     485
English Patient, The (1996)                          481
Scream (1996)                                        478
Toy Story (1995)                                     452
Air Force One (1997)                                 431
Independence Day (ID4) (1996)                        429
Raiders of the Lost Ark (1981)                       420
Godfather, The (1972)                                413
Pulp Fiction (1994)                                  394
Twelve Monkeys (1995)                                392
Silence of the Lambs, The (1991)                     390
Jerry Maguire (1996)                                 384
Chasing Amy (1997)                                   379
Rock, The (1996)               

In [21]:
def favoriteMovies(activeUser,N):
    
    topMovies=pd.DataFrame.sort_values(
        data[data.userId==activeUser],['rating'],ascending=[0])[:N]

    return list(topMovies.title)


In [16]:
userItemRatingMatrix=pd.pivot_table(data, values='rating',
                                    index=['userId'], columns=['itemId'])
userItemRatingMatrix.head()

itemId,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
def similarity(user1,user2):
    user1=np.array(user1)-np.nanmean(user1) # we are first normalizing user1 by 
    
    user2=np.array(user2)-np.nanmean(user2)
    
    commonItemIds=[i for i in range(len(user1)) if user1[i]>0 and user2[i]>0]
    if len(commonItemIds)==0:
        return 0
    else:
        user1=np.array([user1[i] for i in commonItemIds])
        user2=np.array([user2[i] for i in commonItemIds])
        return correlation(user1,user2)

In [19]:
def nearestNeighbourRatings(activeUser,K):
    
    similarityMatrix=pd.DataFrame(index=userItemRatingMatrix.index,
                                  columns=['Similarity'])
    
    for i in userItemRatingMatrix.index:
        
        similarityMatrix.loc[i]=similarity(userItemRatingMatrix.loc[activeUser],
                                          userItemRatingMatrix.loc[i])
        
    similarityMatrix=pd.DataFrame.sort_values(similarityMatrix,
                                              ['Similarity'],ascending=[0])
    nearestNeighbours=similarityMatrix[:K]
    
    neighbourItemRatings=userItemRatingMatrix.loc[nearestNeighbours.index]
    
    predictItemRating=pd.DataFrame(index=userItemRatingMatrix.columns, columns=['Rating'])
    
    for i in userItemRatingMatrix.columns:
        
        predictedRating=np.nanmean(userItemRatingMatrix.loc[activeUser])
        
        for j in neighbourItemRatings.index:
            
            if userItemRatingMatrix.loc[j,i]>0:
                
                predictedRating += (userItemRatingMatrix.loc[j,i]
                                    -np.nanmean(userItemRatingMatrix.loc[j]))*nearestNeighbours.loc[j,'Similarity']
        
        predictItemRating.loc[i,'Rating']=predictedRating
    return predictItemRating

def topNRecommendations(activeUser,N):
    predictItemRating=nearestNeighbourRatings(activeUser,10)
    
    moviesAlreadyWatched=list(userItemRatingMatrix.loc[activeUser]
                              .loc[userItemRatingMatrix.loc[activeUser]>0].index)
   
    predictItemRating=predictItemRating.drop(moviesAlreadyWatched)
    topRecommendations=pd.DataFrame.sort_values(predictItemRating,
                                                ['Rating'],ascending=[0])[:N]
    
    topRecommendationTitles=(movieInfo.loc[movieInfo.itemId.isin(topRecommendations.index)])
    return list(topRecommendationTitles.title)

In [23]:
activeUser=5
print("Top 5 Favourite Movies :-\n")
print(favoriteMovies(activeUser,5))
print("\nTop 5 Recommended Movies :-\n")
print(topNRecommendations(activeUser,5))

Top 5 Favourite Movies :-

['Men in Black (1997)', 'Blade Runner (1982)', 'Empire Strikes Back, The (1980)', 'Wrong Trousers, The (1993)', 'Blues Brothers, The (1980)']

Top 5 Recommended Movies :-



/Users/harshilyadav/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:543: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(um, vm) / (norm(um) * norm(vm))


['Truth About Cats & Dogs, The (1996)', 'Sense and Sensibility (1995)', 'Scream (1996)', 'L.A. Confidential (1997)', 'First Wives Club, The (1996)']


In [24]:
def matrixFactorization(R, K, steps=10, gamma=0.001,lamda=0.02):
   
    N=len(R.index)# Number of users
    M=len(R.columns) # Number of items 
    P=pd.DataFrame(np.random.rand(N,K),index=R.index)
    
    Q=pd.DataFrame(np.random.rand(M,K),index=R.columns)
    
    for step in range(steps):
        
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    
                    e= e + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
        print(step)
    return P,Q


(P,Q)=matrixFactorization(userItemRatingMatrix.iloc[:100,:100],K=2,gamma=0.001,lamda=0.02, steps=100)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [31]:
activeUser=5
predictItemRating=pd.DataFrame(np.dot(P.loc[activeUser],Q.T),index=Q.index,columns=['Rating'])

topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])[:5]

topRecommendationTitles=movieInfo.loc[movieInfo.itemId.isin(topRecommendations.index)]
print(list(topRecommendationTitles.title))


['Usual Suspects, The (1995)', 'Star Wars (1977)', 'Three Colors: Red (1994)', 'Shawshank Redemption, The (1994)', 'Fargo (1996)']


In [27]:
allitems=[]

def support(itemset):
    userList=userItemRatingMatrix.index
    nUsers=len(userList)
    ratingMatrix=userItemRatingMatrix
    for item in itemset:
        ratingMatrix=ratingMatrix.loc[ratingMatrix.loc[:,item]>0]
        userList=ratingMatrix.index
    
    return float(len(userList))/float(nUsers)

minsupport=0.3
for item in list(userItemRatingMatrix.columns):
    itemset=[item]
    if support(itemset)>minsupport:
        allitems.append(item)

In [28]:
len(allitems)

47

In [29]:
minconfidence=0.1
assocRules=[]
i=2
for rule in itertools.permutations(allitems,2):
    
    from_item=[rule[0]]
    to_item=rule
     
    confidence=support(to_item)/support(from_item)
    if confidence>minconfidence and support(to_item)>minsupport:
        assocRules.append(rule)

In [30]:
assocRules

[(1, 50),
 (1, 100),
 (1, 117),
 (1, 121),
 (1, 181),
 (7, 50),
 (7, 100),
 (7, 181),
 (50, 1),
 (50, 7),
 (50, 56),
 (50, 69),
 (50, 79),
 (50, 98),
 (50, 100),
 (50, 117),
 (50, 121),
 (50, 127),
 (50, 172),
 (50, 173),
 (50, 174),
 (50, 181),
 (50, 204),
 (50, 210),
 (50, 222),
 (50, 237),
 (50, 258),
 (50, 288),
 (50, 294),
 (50, 405),
 (56, 50),
 (56, 98),
 (56, 100),
 (56, 174),
 (56, 181),
 (69, 50),
 (79, 50),
 (79, 174),
 (98, 50),
 (98, 56),
 (98, 100),
 (98, 174),
 (98, 181),
 (100, 1),
 (100, 7),
 (100, 50),
 (100, 56),
 (100, 98),
 (100, 117),
 (100, 121),
 (100, 127),
 (100, 174),
 (100, 181),
 (100, 237),
 (117, 1),
 (117, 50),
 (117, 100),
 (117, 121),
 (117, 181),
 (121, 1),
 (121, 50),
 (121, 100),
 (121, 117),
 (121, 181),
 (121, 405),
 (127, 50),
 (127, 100),
 (127, 181),
 (172, 50),
 (172, 174),
 (172, 181),
 (173, 50),
 (174, 50),
 (174, 56),
 (174, 79),
 (174, 98),
 (174, 100),
 (174, 172),
 (174, 181),
 (174, 204),
 (174, 210),
 (181, 1),
 (181, 7),
 (181, 50),
